In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
mypath = '/content/drive/MyDrive/data/MSBC 5680 (Optimization  Modeling)'
os.chdir(mypath)

In [3]:
!pip install gurobipy
from typing import List
import gurobipy as gp
from gurobipy import GRB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 20.4 MB/s eta 0:00:00


In [4]:
# Parameters

employees = ['M', 'A', 'B', 'C', 'D'] # 'M' is manager (salary position) / 'A', 'B', 'C', and 'D' are sales associates (hourly positions)
# Manager is included in shifts/schedule, but excluded from Obj Function (hourly position payroll)

start_times = [10, 11, 12, 13, 14, 15, 16, 17] # Possible shift start times (10am - 5pm)

duration = [3, 4, 5, 6, 7, 8, 9, 10] # Possible shift lengths (hours)

days = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'] # For optimizing a weekly schedule

hours = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19] # Store hours that need to be covered (10am-11am, 11am-12pm, 12pm-1pm, ..., 7pm-8pm)
# Every day has the same store operational hours

shifts = [] # Possible start time/duration combinations for shifts
for start_time in start_times:
    for dur in duration:
        if start_time + dur <= max(hours) + 1: # Ensure shifts stay within store hours
            shifts.append((start_time, dur))

reg_rate = 15 # Regular pay rate for all hourly positions ($15/hr). For weekly hours worked up to 40
ot_rate = reg_rate * 1.5  # Overtime pay rate (time and a half) for all hourly positions ($22.50/hr). For weekly hours worked over 40

full_time_employees = ['A', 'B'] # Full-time employees ('A' and 'B') have to be scheduled to work at least 20 hours per week
part_time_employees = ['C', 'D'] # Part-time employees ('C' and 'D') can't be scheduled to work over 20 hours per week

anticipated_volume = { # Anticipated volume of shoppers/customers for each day ('High', 'Medium', or 'Low')
    'Mon': 'high', # Based on domain knowledge from previous experience (weekends, pay-days, sales, weather, etc.)
    'Tues': 'medium', # High volume days will need more sales associates scheduled to capitalize on sales opportunites
    'Wed': 'low', # Low volume days will need less less sales associates scheduled to reduce the opportunity cost (available scheduling hours, payroll, etc.)
    'Thurs': 'low',
    'Fri': 'medium',
    'Sat': 'high',
    'Sun': 'high'
}

demand = {day: [1] * len(hours) for day in days} # Create dictionary with baseline of 1 (1 employee must be scheduled for each operational hour)
# Demand represents how many sales associates should be coving the floor at the same time (for each hour of each day)

for day, volume in anticipated_volume.items():
    base_demand = 1 # Reassuring a minimum of single coverage at all times
    if volume == 'high': # High volume days need a minimum of triple coverage (one less than the ideal coverage)
        base_demand = 2
    elif volume == 'medium': # Medium volume days need a minimum of double coverage (one less than ideal coverage)
        base_demand = 1

    demand[day] = [base_demand] * len(hours) # Expand daily minimum coverages over all operational hours

import random

random.seed(1001)

for day, volume in anticipated_volume.items():
    if volume == 'high': # High volume days should ideally have triple coverage
        for hour in random.sample(hours, 8): # Not always possible, so manager can adjust the number of hours that receive triple coverage
                                              # Hours could be selected strategically/manually instead of randomly in application
            demand[day][hour - min(hours)] = 3
    elif volume == 'medium':
        for hour in random.sample(hours, 8):
            demand[day][hour - min(hours)] = 2

marketing_days = ['Tues', 'Thurs'] # Out-of-store maketing needs to be done at least twice per week
marketing_hours = {'Tues': 11, 'Thurs': 14} # Days and times can be updated based on scheduled marketing events

for day in marketing_days:
    start_hour = marketing_hours[day]
    for i in range(4): # Plan on marketing taking 4 hours
        demand[day][start_hour - min(hours) + i] += 1 # One extra employee needs to be added for marketing days/times to preserve store coverage

In [5]:
m = gp.Model("Scheduling")

# Decision Variables
x = m.addVars(employees, shifts, days, vtype=gp.GRB.BINARY)
reg_hours = m.addVars(employees)
ot_hours = m.addVars(employees)

# Objective Function
non_manager_employees = [e for e in employees if e != 'M'] # Managers not included in payroll cost when it comes to store scheduling variability
m.setObjective(gp.quicksum(reg_hours[e] * reg_rate + ot_hours[e] * ot_rate for e in non_manager_employees), gp.GRB.MINIMIZE)
# Minimize weekly payroll cost for hourly positions. Account for regular vs overtime pay

# Constraints

# Demand Constraint
for d in days:
    for h in hours:
        employee_count = gp.LinExpr()
        for e in employees:
            for start_time, dur in shifts:
                if start_time <= h and start_time + dur > h: # For each hour of each day, if an employees shift covers that time...
                    employee_count += x[e, start_time, dur, d] # ...count that employee in the total employee count for that time
        m.addConstr(employee_count >= demand[d][h - min(hours)], f"Demand_Constraint_{d}_{h}") # Employee counts must be meet the corresponding demands

# Regular/Overtime Hours Constraints
for e in employees:
    m.addConstr(
        sum(s[1] * x[e, s[0], s[1], d] for s in shifts for d in days) == reg_hours[e] + ot_hours[e]
    )

for e in employees:
  m.addConstr(reg_hours[e] <= 40) # Regular weekly hours can't be over 40

# One Shift Per Day Constraint
for e in employees: # An employee can only work one shift per day
    for d in days:
        m.addConstr(gp.quicksum(x[e, s[0], s[1], d] for s in shifts) <= 1, f"One_Shift_Per_Day_{e}_{d}")

# Full-time/Part-time Constraints
for e in full_time_employees:  # Full-time employees must work over 20 hours per week
    m.addConstr(sum(s[1] * x[e, s[0], s[1], d] for s in shifts for d in days) >= 20, f"Min_Hours_FullTime_{e}")

for e in part_time_employees: # Part-time employees can't work over 20 hours per week
    m.addConstr(sum(s[1] * x[e, s[0], s[1], d] for s in shifts for d in days) <= 20, f"Max_Hours_PartTime_{e}")

# Maximum Manager Hours Constraint
m.addConstr(sum(s[1] * x['M', s[0], s[1], d] for s in shifts for d in days) <= 50, "Max_Hours_Manager_M") # Manager doesn't want to work over 50 hours per week

# Maximum Days Scheduled Constraint
for e in employees:
    m.addConstr(sum(x[e, s[0], s[1], d] for s in shifts for d in days) <= 5, f"Max_5_Working_Days_{e}") # All employees need to have at least 2 days of per week

# Employee Availability Constraints
for s in shifts:
    m.addConstr(x['A', s[0], s[1], 'Sat'] == 0, "Availability_A_Sat") # Employee A needs Saturdays off

for s in shifts:
    if s[0] > 16:
        m.addConstr(x['C', s[0], s[1], 'Mon'] == 0, "Availability_C_Mon_Eve") # Employee C needs Monday evenings off (after 4pm)
        m.addConstr(x['C', s[0], s[1], 'Wed'] == 0, "Availability_C_Wed_Eve") # Employee C needs Wednesday evenings off (after 4pm)

for s in shifts:
    m.addConstr(x['D', s[0], s[1], 'Sun'] == 0, "Availability_D_Sun") # Employee D cannot work on Sundays



Restricted license - for non-production use only - expires 2025-11-24


In [6]:
m.optimize()
print()

if m.status == gp.GRB.OPTIMAL:
    print("Total Cost of the Schedule:", m.objVal)
else:
    print("No optimal solution found. Status code:", m.status)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 199 rows, 1270 columns and 11849 nonzeros
Model fingerprint: 0x3dadeaa4
Variable types: 10 continuous, 1260 integer (1260 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 101 rows and 443 columns
Presolve time: 0.04s
Presolved: 98 rows, 827 columns, 6638 nonzeros
Variable types: 4 continuous, 823 integer (821 binary)
Found heuristic solution: objective 2250.0000000

Root relaxation: objective 1.500000e+03, 259 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [7]:
def format_time(hour):
    if hour == 0 or hour == 24:
        return "12am"
    elif hour == 12:
        return "12pm"
    elif hour < 12:
        return f"{hour}am"
    else:
        return f"{hour-12}pm"

print("Weekly Schedule")
print()

for d in days:
    print(f'Day: {d}')
    print('--------------------------------')
    for e in employees:
        shifts_assigned = False
        for s in shifts:
            if x[e, s[0], s[1], d].X > 0:
                end_time = s[0] + s[1]
                formatted_start = format_time(s[0])
                formatted_end = format_time(end_time)
                print(f'Employee {e}: {formatted_start} - {formatted_end} ({s[1]} hours)')
                shifts_assigned = True
    print()


Weekly Schedule

Day: Mon
--------------------------------
Employee M: 10am - 8pm (10 hours)
Employee A: 5pm - 8pm (3 hours)
Employee B: 10am - 4pm (6 hours)
Employee C: 10am - 7pm (9 hours)

Day: Tues
--------------------------------
Employee M: 10am - 8pm (10 hours)
Employee A: 10am - 3pm (5 hours)
Employee B: 12pm - 8pm (8 hours)

Day: Wed
--------------------------------
Employee M: 10am - 8pm (10 hours)

Day: Thurs
--------------------------------
Employee A: 10am - 8pm (10 hours)
Employee C: 2pm - 6pm (4 hours)

Day: Fri
--------------------------------
Employee A: 4pm - 8pm (4 hours)
Employee B: 10am - 3pm (5 hours)
Employee D: 10am - 8pm (10 hours)

Day: Sat
--------------------------------
Employee M: 10am - 8pm (10 hours)
Employee B: 1pm - 8pm (7 hours)
Employee C: 10am - 1pm (3 hours)
Employee D: 10am - 6pm (8 hours)

Day: Sun
--------------------------------
Employee M: 10am - 8pm (10 hours)
Employee A: 10am - 4pm (6 hours)
Employee B: 10am - 7pm (9 hours)
Employee C: 5pm -

In [8]:
print("Total Hours Summary")
print()

for e in employees:
    print()
    total_hours = sum(s[1] * x[e, s[0], s[1], d].X for s in shifts for d in days)
    print(f'Employee {e}:')
    print(f'Total Scheduled Hours: {total_hours}')
    if e != 'M':
      print(f'  Regular Hours: {reg_hours[e].X}')
      print(f'  Overtime Hours: {ot_hours[e].X}')


Total Hours Summary


Employee M:
Total Scheduled Hours: 50.0

Employee A:
Total Scheduled Hours: 28.0
  Regular Hours: 28.0
  Overtime Hours: 0.0

Employee B:
Total Scheduled Hours: 35.0
  Regular Hours: 35.0
  Overtime Hours: 0.0

Employee C:
Total Scheduled Hours: 19.0
  Regular Hours: 19.0
  Overtime Hours: 0.0

Employee D:
Total Scheduled Hours: 18.0
  Regular Hours: 18.0
  Overtime Hours: 0.0


In [9]:
print("Employee Schedule per Hour")
print()

for d in days:
    print(f'{d}:')
    for h in hours:
        employee_count = 0
        for e in employees:
            for s in shifts:
                if s[0] <= h < s[0] + s[1] and x[e, s[0], s[1], d].X > 0:
                    employee_count += 1
        formatted_hour = format_time(h)
        print(f' {formatted_hour}: {employee_count} employees scheduled')
    print('--------------------------------')


Employee Schedule per Hour

Mon:
 10am: 3 employees scheduled
 11am: 3 employees scheduled
 12pm: 3 employees scheduled
 1pm: 3 employees scheduled
 2pm: 3 employees scheduled
 3pm: 3 employees scheduled
 4pm: 2 employees scheduled
 5pm: 3 employees scheduled
 6pm: 3 employees scheduled
 7pm: 2 employees scheduled
--------------------------------
Tues:
 10am: 2 employees scheduled
 11am: 2 employees scheduled
 12pm: 3 employees scheduled
 1pm: 3 employees scheduled
 2pm: 3 employees scheduled
 3pm: 2 employees scheduled
 4pm: 2 employees scheduled
 5pm: 2 employees scheduled
 6pm: 2 employees scheduled
 7pm: 2 employees scheduled
--------------------------------
Wed:
 10am: 1 employees scheduled
 11am: 1 employees scheduled
 12pm: 1 employees scheduled
 1pm: 1 employees scheduled
 2pm: 1 employees scheduled
 3pm: 1 employees scheduled
 4pm: 1 employees scheduled
 5pm: 1 employees scheduled
 6pm: 1 employees scheduled
 7pm: 1 employees scheduled
--------------------------------
Thurs:
 